In [1]:
from tools.data import get_datasets

train, test = get_datasets()

Reading data file


In [4]:
# Exclusion des -1 qui sont majoritaires dans les datasets
import numpy as np

train = np.compress(train[:,-1] >= 0, train, axis=0)
test = np.compress(test[:,-1] >= 0, test, axis=0)

## Test d'un classifieur k ppv avec des features simples

Dans la suite on lance un classifieur 5-ppv en utilisant 3 features: les std dev des séries des 3 capteurs.

### discussion des résultats
Les résultats sur le test set sont des prédictions justes dans 11% des cas.

En choisissant au pif on aurait bon dans $\frac{1}{\text{nombre de classes}} = \frac{1}{11} = 0.9$%.

Donc avec ce classifieur tout simple on est un tout petit peu mieux que le pif... encourageant non ?

In [5]:
# Create std dev features for all 3 sensors
import numpy as np

def create_std_dev(dataset):
    features = np.zeros((dataset.shape[0], 5))
    for i in range(dataset.shape[0]):
        length = dataset[i,0]
        features[i,0] = np.std(dataset[i, 1+512*0:1+512*0+length]) 
        features[i,1] = np.std(dataset[i, 1+512*1:1+512*1+length]) 
        features[i,2] = np.std(dataset[i, 1+512*2:1+512*2+length])
        features[i,3] = np.std(dataset[i, 1+512*3:1+512*3+length])
        features[i,4] = dataset[i,-1]
    return features

train_std = create_std_dev(train)
test_std = create_std_dev(test)

/home/python/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:8: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/python/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:9: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/python/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:10: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/python/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:11: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [6]:
from sklearn.neighbors import KNeighborsClassifier

# train set
neigh = KNeighborsClassifier(n_neighbors=5)
neigh.fit(train_std[:,:-1], train_std[:,-1])

# test set
total_correct = 0
for i in range(test_std.shape[0]):
    if neigh.predict(test_std[i,:-1])[0] == test_std[i,-1]:
        total_correct += 1
        
print 'Pourcentage correct:', float(total_correct) / test_std.shape[0] * 100

Pourcentage correct: 9.51198311188
